# Example Model Servers with Seldon

## Setup Seldon Core

Use the setup notebook to [Setup Cluster](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html#Setup-Cluster) with [Ambassador Ingress](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html#Ambassador) and [Seldon Core](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html#Install-Seldon-Core). Instructions [also online](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html).

In [1]:
!kubectl create namespace seldon

Error from server (AlreadyExists): namespaces "seldon" already exists


In [16]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

Context "kind-kind" modified.


In [3]:
import json

## Serve SKLearn Iris Model

In order to deploy SKLearn artifacts, we can leverage the [pre-packaged SKLearn inference server](https://docs.seldon.io/projects/seldon-core/en/latest/servers/sklearn.html).
The exposed API can follow either:

- The default Seldon protocol. 
- The [KFServing V2 protocol](https://docs.seldon.io/projects/seldon-core/en/latest/servers/sklearn.html##v2-kfserving-protocol-incubating).

For details on each of these protocols, you can check the [documentation section on API protocols](https://docs.seldon.io/projects/seldon-core/en/latest/graph/protocols.html#v2-kfserving-protocol).


### Default Seldon protocol

To deploy and start serving an SKLearn artifact using Seldon's default protocol, we can use a config like the one below:

In [25]:
%%writefile ../servers/sklearnserver/samples/iris.yaml
apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  name: sklearn
spec:
  name: iris
  predictors:
  - graph:
      children: []
      implementation: SKLEARN_SERVER
      modelUri: gs://seldon-models/sklearn/iris
      name: classifier
    name: default
    replicas: 1
    svcOrchSpec: 
      env: 
      - name: SELDON_LOG_LEVEL
        value: DEBUG

Overwriting ../servers/sklearnserver/samples/iris.yaml


We can then apply it to deploy it to our Kubernetes cluster.

In [26]:
!kubectl apply -f ../servers/sklearnserver/samples/iris.yaml

seldondeployment.machinelearning.seldon.io/sklearn created


In [27]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=sklearn -o jsonpath='{.items[0].metadata.name}')

deployment "sklearn-default-0-classifier" successfully rolled out


Once it's deployed we can send our sklearn model requests

#### REST Requests

In [28]:
X=!curl -s -d '{"data": {"ndarray":[[1.0, 2.0, 5.0, 6.0]]}}' \
   -X POST http://localhost:8003/seldon/seldon/sklearn/api/v1.0/predictions \
   -H "Content-Type: application/json"
d=json.loads(X[0])
print(d)

{'data': {'names': ['t:0', 't:1', 't:2'], 'ndarray': [[9.912315378486718e-07, 0.0007015931307743852, 0.9992974156376878]]}, 'meta': {}}


In [29]:
from seldon_core.seldon_client import SeldonClient
sc = SeldonClient(deployment_name="sklearn",namespace="seldon")

In [30]:
r = sc.predict(gateway="ambassador",transport="rest",shape=(1,4))
print(r)
assert(r.success==True)

Success:True message:
Request:
meta {
}
data {
  tensor {
    shape: 1
    shape: 4
    values: 0.6806334106409295
    values: 0.28830360482888695
    values: 0.8975654688578534
    values: 0.24493223527153596
  }
}

Response:
{'data': {'names': ['t:0', 't:1', 't:2'], 'tensor': {'shape': [1, 3], 'values': [0.14449927217770223, 0.5104835080299303, 0.34501721979236755]}}, 'meta': {}}


#### gRPC Requests

In [31]:
r = sc.predict(gateway="ambassador",transport="grpc",shape=(1,4))
print(r)
assert(r.success==True)

Success:True message:
Request:
{'meta': {}, 'data': {'tensor': {'shape': [1, 4], 'values': [0.22157381246204433, 0.020827790786912548, 0.7633031884959527, 0.19217598516001178]}}}
Response:
{'meta': {}, 'data': {'names': ['t:0', 't:1', 't:2'], 'tensor': {'shape': [1, 3], 'values': [0.10758420949891327, 0.4796849750770796, 0.4127308154240072]}}}


In [32]:
X=!cd ../executor/proto && grpcurl -d '{"data":{"ndarray":[[1.0,2.0,5.0,6.0]]}}' \
         -rpc-header seldon:sklearn -rpc-header namespace:seldon \
         -plaintext \
         -proto ./prediction.proto  0.0.0.0:8003 seldon.protos.Seldon/Predict
d=json.loads("".join(X))
print(d)

{'meta': {}, 'data': {'names': ['t:0', 't:1', 't:2'], 'ndarray': [[9.912315378486718e-07, 0.0007015931307743852, 0.9992974156376878]]}}


And delete the model we deployed

In [33]:
!kubectl delete -f ../servers/sklearnserver/samples/iris.yaml

seldondeployment.machinelearning.seldon.io "sklearn" deleted


### KFServing V2 protocol

We can deploy a SKLearn artifact, exposing an API compatible with [KFServing's V2 Protocol](https://docs.seldon.io/projects/seldon-core/en/latest/servers/sklearn.html##v2-kfserving-protocol-incubating) by specifying the `protocol` of our `SeldonDeployment` as `kfserving`.
For example, we can consider the config below:

In [1]:
%%writefile resources/iris-sklearn-v2.yaml
apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: sklearn
spec:
  name: iris
  protocol: kfserving
  predictors:
  - graph:
      children: []
      implementation: SKLEARN_SERVER
      modelUri: gs://seldon-models/sklearn/iris
      name: classifier
    name: default
    replicas: 1

Writing resources/iris-sklearn-v2.yaml


We can then apply it to deploy our model to our Kubernetes cluster.

In [2]:
!kubectl apply -f resources/iris-sklearn-v2.yaml

seldondeployment.machinelearning.seldon.io/sklearn created


In [3]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=sklearn -o jsonpath='{.items[0].metadata.name}')

Waiting for deployment "sklearn-default-0-classifier" rollout to finish: 0 of 1 updated replicas are available...
deployment "sklearn-default-0-classifier" successfully rolled out


Once it's deployed, we can send inference requests to our model.
Note that, since it's using the KFServing's V2 Protocol, these requests will be different to the ones using the default Seldon Protocol.

In [4]:
import requests
import json

inference_request = {
    "inputs": [
        {
          "name": "predict",
          "shape": [1, 4],
          "datatype": "FP32",
          "data": [[1, 2, 3, 4]]
        }
    ]
}

endpoint = "http://localhost:8003/seldon/seldon/sklearn/v2/models/infer"
response = requests.post(endpoint, json=inference_request)

print(json.dumps(response.json(), indent=2))
assert response.ok

{
  "model_name": "classifier",
  "model_version": "v1",
  "id": "5a0bce13-98e8-4588-9fc5-7c8b11140174",
  "parameters": null,
  "outputs": [
    {
      "name": "predict",
      "shape": [
        1
      ],
      "datatype": "FP32",
      "parameters": null,
      "data": [
        2
      ]
    }
  ]
}


Finally, we can delete the model we deployed.

In [5]:
!kubectl delete -f resources/iris-sklearn-v2.yaml

seldondeployment.machinelearning.seldon.io "sklearn" deleted


## Serve XGBoost Iris Model

In order to deploy XGBoost models, we can leverage the [pre-packaged XGBoost inference server](https://docs.seldon.io/projects/seldon-core/en/latest/servers/xgboost.html).
The exposed API can follow either:

- The default Seldon protocol. 
- The [KFServing V2 protocol](https://docs.seldon.io/projects/seldon-core/en/latest/servers/xgboost.html##v2-kfserving-protocol-incubating).

For details on each of these protocols, you can check the [documentation section on API protocols](https://docs.seldon.io/projects/seldon-core/en/latest/graph/protocols.html#v2-kfserving-protocol).

### Default Seldon protocol

We can deploy a XGBoost model uploaded to an object store by using the XGBoost model server implementation as shown in the config below:

In [18]:
%%writefile ../servers/xgboostserver/samples/iris.yaml
apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: xgboost
spec:
  name: iris
  predictors:
  - graph:
      children: []
      implementation: XGBOOST_SERVER
      modelUri: gs://seldon-models/xgboost/iris
      name: classifier
    name: default
    replicas: 1

Overwriting ../servers/xgboostserver/samples/iris.yaml


And then we apply it to deploy it to our kubernetes cluster

In [19]:
!kubectl apply -f ../servers/xgboostserver/samples/iris.yaml

seldondeployment.machinelearning.seldon.io/xgboost created


In [20]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=xgboost -o jsonpath='{.items[0].metadata.name}')

deployment "xgboost-default-0-classifier" successfully rolled out


#### Rest Requests

In [23]:
X=!curl -s -d '{"data": {"ndarray":[[1.0, 2.0, 5.0, 6.0]]}}' \
   -X POST http://localhost:8003/seldon/seldon/xgboost/api/v1.0/predictions \
   -H "Content-Type: application/json"
d=json.loads(X[0])
print(d)

{'data': {'names': [], 'ndarray': [2.0]}, 'meta': {}}


In [24]:
from seldon_core.seldon_client import SeldonClient
sc = SeldonClient(deployment_name="xgboost",namespace="seldon")

In [25]:
r = sc.predict(gateway="ambassador",transport="rest",shape=(1,4))
print(r)
assert(r.success==True)

Success:True message:
Request:
meta {
}
data {
  tensor {
    shape: 1
    shape: 4
    values: 0.9816489970625223
    values: 0.15136579121850824
    values: 0.9369540193482753
    values: 0.6637451146599143
  }
}

Response:
{'data': {'names': [], 'tensor': {'shape': [1], 'values': [0.0]}}, 'meta': {}}


#### gRPC Requests

In [26]:
r = sc.predict(gateway="ambassador",transport="grpc",shape=(1,4))
print(r)
assert(r.success==True)

Success:True message:
Request:
{'meta': {}, 'data': {'tensor': {'shape': [1, 4], 'values': [0.3316136646069059, 0.35708074352350294, 0.7611407638250924, 0.9509923468248295]}}}
Response:
{'meta': {}, 'data': {'tensor': {'shape': [1], 'values': [0.0]}}}


In [27]:
X=!cd ../executor/proto && grpcurl -d '{"data":{"ndarray":[[1.0,2.0,5.0,6.0]]}}' \
         -rpc-header seldon:xgboost -rpc-header namespace:seldon \
         -plaintext \
         -proto ./prediction.proto  0.0.0.0:8003 seldon.protos.Seldon/Predict
d=json.loads("".join(X))
print(d)

{'meta': {}, 'data': {'ndarray': [2]}}


And delete the model we deployed

In [28]:
!kubectl delete -f ../servers/xgboostserver/samples/iris.yaml

seldondeployment.machinelearning.seldon.io "xgboost" deleted


### KFServing V2 protocol

We can deploy a XGBoost model, exposing an API compatible with [KFServing's V2 Protocol](https://docs.seldon.io/projects/seldon-core/en/latest/servers/xgboost.html##v2-kfserving-protocol-incubating) by specifying the `protocol` of our `SeldonDeployment` as `kfserving`.
For example, we can consider the config below:

In [29]:
%%writefile ../testing/resources/iris-xgboost-v2.yaml
apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: xgboost
spec:
  name: iris
  protocol: kfserving
  predictors:
  - graph:
      children: []
      implementation: XGBOOST_SERVER
      modelUri: gs://seldon-models/xgboost/iris
      name: classifier
    name: default
    replicas: 1

Overwriting ../testing/resources/iris-xgboost-v2.yaml


We can then apply it to deploy our model to our Kubernetes cluster.

In [30]:
!kubectl apply -f ../testing/resources/iris-xgboost-v2.yaml

seldondeployment.machinelearning.seldon.io/xgboost created


In [31]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=xgboost -o jsonpath='{.items[0].metadata.name}')

Waiting for deployment "xgboost-default-0-classifier" rollout to finish: 0 of 1 updated replicas are available...
deployment "xgboost-default-0-classifier" successfully rolled out


Once it's deployed, we can send inference requests to our model.
Note that, since it's using the KFServing's V2 Protocol, these requests will be different to the ones using the default Seldon Protocol.

In [34]:
import requests
import json

inference_request = {
    "inputs": [
        {
          "name": "predict",
          "shape": [1, 4],
          "datatype": "FP32",
          "data": [[1, 2, 3, 4]]
        }
    ]
}

endpoint = "http://localhost:8003/seldon/seldon/xgboost/v2/models/infer"
response = requests.post(endpoint, json=inference_request)

print(json.dumps(response.json(), indent=2))
assert response.ok

{
  "model_name": "classifier",
  "model_version": "v1",
  "id": "7b3fc4b1-bba1-4119-b19a-53649bb7b885",
  "parameters": null,
  "outputs": [
    {
      "name": "predict",
      "shape": [
        1
      ],
      "datatype": "FP32",
      "parameters": null,
      "data": [
        2.0
      ]
    }
  ]
}


Finally, we can delete the model we deployed.

In [35]:
!kubectl delete -f ../testing/resources/iris-xgboost-v2.yaml

seldondeployment.machinelearning.seldon.io "xgboost" deleted


## Serve Tensorflow MNIST Model
We can deploy a tensorflow model uploaded to an object store by using the
tensorflow model server implementation as the config below.

This notebook contains two examples, one which shows how you can use the
TFServing prepackaged serve with the Seldon Protocol, and a second one which
shows how you can deploy it using the tensorlfow protocol (so you can send
requests of the exact format as you would to a tfserving server).

### Serve Tensorflow MNIST Model with Seldon Protocol

The config file below shows how you can deploy your Tensorflow model which
exposes the Seldon protocol.

In [66]:
%%writefile ../servers/tfserving/samples/mnist_rest.yaml
apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  name: tfserving
spec:
  name: mnist
  predictors:
  - graph:
      children: []
      implementation: TENSORFLOW_SERVER
      modelUri: gs://seldon-models/tfserving/mnist-model
      name: mnist-model
      parameters:
        - name: signature_name
          type: STRING
          value: predict_images
        - name: model_name
          type: STRING
          value: mnist-model
        - name: model_input
          type: STRING
          value: images
        - name: model_output
          type: STRING
          value: scores     
    name: default
    replicas: 1

Overwriting ../servers/tfserving/samples/mnist_rest.yaml


In [67]:
!kubectl apply -f ../servers/tfserving/samples/mnist_rest.yaml

seldondeployment.machinelearning.seldon.io/tfserving created


In [68]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=tfserving -o jsonpath='{.items[0].metadata.name}')

deployment "tfserving-default-0-mnist-model" successfully rolled out


In [69]:
from seldon_core.seldon_client import SeldonClient
sc = SeldonClient(deployment_name="tfserving",namespace="seldon")

#### REST Request

In [70]:
r = sc.predict(gateway="ambassador",transport="rest",shape=(1,784))
print(r)
assert(r.success==True)

Success:True message:
Request:
meta {
}
data {
  tensor {
    shape: 1
    shape: 784
    values: 0.7363034329885751
    values: 0.1214752590440702
    values: 0.6811387469595884
    values: 0.6532280226091893
    values: 0.5409643758278618
    values: 0.16215988725534547
    values: 0.35479123281330016
    values: 0.5607871182857017
    values: 0.6010761255325676
    values: 0.8902670778749476
    values: 0.0868570731397954
    values: 0.7904570127547179
    values: 0.7122542998518603
    values: 0.24630823029939797
    values: 0.06771939750935685
    values: 0.822122604510554
    values: 0.5422718667705746
    values: 0.0818978219637192
    values: 0.5768929648326077
    values: 0.5966960739647604
    values: 0.38243947650220445
    values: 0.1564417664938903
    values: 0.947409102689989
    values: 0.5082818863604522
    values: 0.8921337711444182
    values: 0.9148723260630368
    values: 0.6114620528801846
    values: 0.743021033845883
    values: 0.5153866230874219
    values: 0

#### gRPC Request

In [71]:
r = sc.predict(gateway="ambassador",transport="grpc",shape=(1,784))
print(r)
assert(r.success==True)

Success:True message:
Request:
{'meta': {}, 'data': {'tensor': {'shape': [1, 784], 'values': [0.13419748086376349, 0.23178703054633498, 0.44223572424534485, 0.2696273680588255, 0.07152377975849233, 0.5727816756904325, 0.39309147982375336, 0.3721411402484911, 0.551244898235985, 0.43975687243817896, 0.8415874250304912, 0.01724839861848615, 0.9992444311222564, 0.20116248615583598, 0.8134491993637052, 0.7874830465957086, 0.6030254329128775, 0.7379539664731155, 0.9732382705725281, 0.3376471422948141, 0.4450901915855564, 0.4104062891617003, 0.6235508101202012, 0.5280596447975435, 0.30707052711675076, 0.8616168522237181, 0.08748039835534571, 0.6441019299651144, 0.2090627479548135, 0.22655911024713882, 0.26494800039445654, 0.0014838612813646845, 0.257641691814799, 0.31542040987433173, 0.9071529110894798, 0.1368715432376224, 0.30286691205277494, 0.3732340970325615, 0.7286908117093446, 0.013476068707320321, 0.37852914951760863, 0.6529607177918756, 0.46718776396978656, 0.510428270339585, 0.888552

And delete the model we deployed

In [72]:
!kubectl delete -f ../servers/tfserving/samples/mnist_rest.yaml

seldondeployment.machinelearning.seldon.io "tfserving" deleted


### Serve Tensorflow Model with Tensorflow protocol

The config file below shows how you can deploy your Tensorflow model which
exposes the Tensorflow protocol.

In [73]:
%%writefile ../servers/tfserving/samples/halfplustwo_rest.yaml
apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  name: hpt
spec:
  name: hpt
  protocol: tensorflow
  transport: rest
  predictors:
  - graph:
      children: []
      implementation: TENSORFLOW_SERVER
      modelUri: gs://seldon-models/tfserving/half_plus_two
      name:  halfplustwo
      parameters:
        - name: model_name
          type: STRING
          value: halfplustwo
    name: default
    replicas: 1

Overwriting ../servers/tfserving/samples/halfplustwo_rest.yaml


In [74]:
!kubectl apply -f ../servers/tfserving/samples/halfplustwo_rest.yaml

seldondeployment.machinelearning.seldon.io/hpt created


In [75]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=hpt -o jsonpath='{.items[0].metadata.name}')

deployment "hpt-default-0-halfplustwo" successfully rolled out


In [76]:
import json
X=!curl -s -d '{"instances": [1.0, 2.0, 5.0]}' \
   -X POST http://localhost:8003/seldon/seldon/hpt/v1/models/halfplustwo/:predict \
   -H "Content-Type: application/json"
d=json.loads("".join(X))
print(d)
assert(d["predictions"][0] == 2.5)

{'predictions': [2.5, 3.0, 4.5]}


In [79]:
X=!cd ../executor/proto && grpcurl \
   -d '{"model_spec":{"name":"halfplustwo"},"inputs":{"x":{"dtype": 1, "tensor_shape": {"dim":[{"size": 3}]}, "floatVal" : [1.0, 2.0, 3.0]}}}' \
   -rpc-header seldon:hpt -rpc-header namespace:seldon \
   -plaintext -proto ./prediction_service.proto \
   0.0.0.0:8003 tensorflow.serving.PredictionService/Predict
d=json.loads("".join(X))
print(d)
assert(d["outputs"]["x"]["floatVal"][0] == 2.5)

{'outputs': {'x': {'dtype': 'DT_FLOAT', 'tensorShape': {'dim': [{'size': '3'}]}, 'floatVal': [2.5, 3, 3.5]}}, 'modelSpec': {'name': 'halfplustwo', 'version': '123', 'signatureName': 'serving_default'}}


In [80]:
!kubectl delete -f ../servers/tfserving/samples/halfplustwo_rest.yaml

seldondeployment.machinelearning.seldon.io "hpt" deleted


## Serve MLFlow Elasticnet Wines Model
We can deploy an MLFlow model uploaded to an object store by using the MLFlow
model server implementation as the config below:

In [83]:
%%writefile ../servers/mlflowserver/samples/elasticnet_wine.yaml
apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  name: mlflow
spec:
  name: wines
  predictors:
  - componentSpecs:
    - spec:
        # We are setting high failureThreshold as installing conda dependencies
        # can take long time and we want to avoid k8s killing the container prematurely
        containers:
        - name: classifier
          livenessProbe:
            initialDelaySeconds: 80
            failureThreshold: 200
            periodSeconds: 5
            successThreshold: 1
            httpGet:
              path: /health/ping
              port: http
              scheme: HTTP
          readinessProbe:
            initialDelaySeconds: 80
            failureThreshold: 200
            periodSeconds: 5
            successThreshold: 1
            httpGet:
              path: /health/ping
              port: http
              scheme: HTTP
    graph:
      children: []
      implementation: MLFLOW_SERVER
      modelUri: gs://seldon-models/mlflow/elasticnet_wine
      name: classifier
    name: default
    replicas: 1

Overwriting ../servers/mlflowserver/samples/elasticnet_wine.yaml


In [91]:
!kubectl apply -f ../servers/mlflowserver/samples/elasticnet_wine.yaml

seldondeployment.machinelearning.seldon.io/mlflow created


In [92]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=mlflow -o jsonpath='{.items[0].metadata.name}')

deployment "mlflow-default-0-classifier" successfully rolled out


#### REST requests

In [99]:
X=!curl -s -d '{"data": {"ndarray":[[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,1.1]]}}' \
   -X POST http://localhost:8003/seldon/seldon/mlflow/api/v1.0/predictions \
   -H "Content-Type: application/json"
d=json.loads(X[0])
print(d)

{'data': {'names': [], 'ndarray': [5.275558760255382]}, 'meta': {}}


In [93]:
from seldon_core.seldon_client import SeldonClient
sc = SeldonClient(deployment_name="mlflow",namespace="seldon")

In [95]:
r = sc.predict(gateway="ambassador",transport="rest",shape=(1,11))
print(r)
assert(r.success==True)

Success:True message:
Request:
meta {
}
data {
  tensor {
    shape: 1
    shape: 11
    values: 0.2221219485862146
    values: 0.48347468791705017
    values: 0.792909901124416
    values: 0.8575020679326859
    values: 0.3393108788709739
    values: 0.9524947980687155
    values: 0.4764630378302829
    values: 0.3330827286255519
    values: 0.02818894894121715
    values: 0.20379458867010114
    values: 0.5176887266713507
  }
}

Response:
{'data': {'names': [], 'tensor': {'shape': [1], 'values': [5.223443971007788]}}, 'meta': {}}


#### gRPC Requests

In [100]:
X=!cd ../executor/proto && grpcurl -d '{"data":{"ndarray":[[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,1.1]]}}' \
         -rpc-header seldon:mlflow -rpc-header namespace:seldon \
         -plaintext \
         -proto ./prediction.proto  0.0.0.0:8003 seldon.protos.Seldon/Predict
d=json.loads("".join(X))
print(d)

{'meta': {}, 'data': {'ndarray': [5.275558760255382]}}


In [101]:
r = sc.predict(gateway="ambassador",transport="grpc",shape=(1,11))
print(r)
assert(r.success==True)

Success:True message:
Request:
{'meta': {}, 'data': {'tensor': {'shape': [1, 11], 'values': [0.3862249013971256, 0.41803382328603056, 0.5006540243492418, 0.22082451828445948, 0.25340037683662653, 0.23224919838688673, 0.23156224762583522, 0.7652379449586778, 0.590985787494962, 0.7813227013394188, 0.48042753535482696]}}}
Response:
{'meta': {}, 'data': {'tensor': {'shape': [1], 'values': [5.214988433508114]}}}


In [102]:
!kubectl delete -f ../servers/mlflowserver/samples/elasticnet_wine.yaml

seldondeployment.machinelearning.seldon.io "mlflow" deleted
